In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
#from keras.layers import LSTM, Dropout, Dense, Layer, Conv1D
from keras.models import Sequential


2023-02-26 16:21:40.155220: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 16:21:40.571717: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-26 16:21:40.571754: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-26 16:21:40.642129: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-02-26 16:21:41.925909: W tensorflow/stream_executor/platform/de

In [2]:
import os
os.chdir("..")
from src import *


In [3]:
_df = pd.read_csv("Data/wig20_d.csv")
# df = pd.read_csv("Data/mwig40_d.csv")
# df = pd.read_csv("Data/swig80_d.csv")

_df.drop(range(1000), inplace=True)
_df.reset_index(inplace=True, drop=True)
_df.head()

,Data,Otwarcie,Najwyzszy,Najnizszy,Zamkniecie,Wolumen
0,1997-02-06,1717.2,1779.8,1717.2,1779.8,4435886.0
1,1997-02-07,1778.2,1808.7,1778.2,1799.0,5448243.0
2,1997-02-10,1797.4,1797.4,1780.3,1783.3,6513315.0
3,1997-02-11,1803.3,1832.2,1803.3,1832.0,5146340.0
4,1997-02-12,1824.6,1824.6,1804.9,1810.2,5748398.0


In [4]:
_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6429 entries, 0 to 6428
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Data        6429 non-null   object 
 1   Otwarcie    6429 non-null   float64
 2   Najwyzszy   6429 non-null   float64
 3   Najnizszy   6429 non-null   float64
 4   Zamkniecie  6429 non-null   float64
 5   Wolumen     6429 non-null   float64
dtypes: float64(5), object(1)
memory usage: 301.5+ KB


In [5]:
_df.columns

Index(['Data', 'Otwarcie', 'Najwyzszy', 'Najnizszy', 'Zamkniecie', 'Wolumen'], dtype='object')

In [6]:
data = _df.drop(columns=["Data"]).values.transpose()

In [7]:
data.shape

(5, 6429)

In [8]:
data_pipeline = DataProcess(data, test_ratio=0.2, validation_ratio=0.2, batch_size=32, threshold_fall=-0.007, threshold_rise=0.007)
data_pipeline.run()
_x_train, _y_train, _x_test, _y_test, _x_validation, _y_validation = data_pipeline.get_data()

In [9]:
print("train", _x_train.shape)
print("test", _x_test.shape)
print("validation", _x_validation.shape)


train (3825, 5, 32, 2)
test (1254, 5, 32, 2)
validation (1254, 5, 32, 2)


In [10]:
print("train", _y_train.shape)
print("test", _y_test.shape)
print("validation", _y_validation.shape)


train (3825, 1, 1, 2)
test (1254, 1, 1, 2)
validation (1254, 1, 1, 2)


In [11]:
np.unique(_y_train, return_counts=True)

(array([-1.,  0.,  1.], dtype=float32), array([2296, 2980, 2374]))

In [12]:
model = tf.keras.Sequential()

model.add(tf.keras.Input(shape=_x_train.shape[1:]))
for i in range(7):
    model.add(tf.keras.layers.Conv2D(2, (1, 3), 1, padding="same"))
    model.add(tf.keras.layers.MaxPooling2D(pool_size=(1, 3), strides=(1, 3), padding='same'))

model.add(tf.keras.layers.Dense(12))
model.add(tf.keras.layers.Dense(7))
model.add(tf.keras.layers.Dense(3, activation="softmax"))
model.add(tf.keras.layers.Dense(2))


2023-02-26 16:21:43.541477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-02-26 16:21:43.541823: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-02-26 16:21:43.541861: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (banjaro): /proc/driver/nvidia/version does not exist
2023-02-26 16:21:43.543176: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
model.build(input_shape=_x_train.shape[1:])

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 5, 32, 2)          14        
                                                                 
 max_pooling2d (MaxPooling2D  (None, 5, 11, 2)         0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 5, 11, 2)          14        
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 4, 2)          0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 5, 4, 2)           14        
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 5, 2, 2)          0

In [15]:
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
#               loss=tf.keras.losses.MeanSquaredError())
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss=tf.keras.losses.SparseCategoricalCrossentropy())
# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
#               loss=tf.keras.losses.Poisson())


In [16]:
print(_x_train.shape, _y_train.shape)

(3825, 5, 32, 2) (3825, 1, 1, 2)


In [17]:
model.fit(_x_train, _y_train, epochs=100)

Epoch 1/100


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/bartko/Environments/financial_market_predictor/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/bartko/Environments/financial_market_predictor/lib/python3.10/site-packages/traitlets/config/application.py", line 978, in launch_instance
      app.start()
    File "/home/bartko/Environments/financial_market_predictor/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/bartko/Environments/financial_market_predictor/lib/python3

In [ ]:
model.evaluate(_x_test, _y_test)

In [ ]:
model.evaluate(_x_validation, _y_validation)

In [ ]:
# model.save("Models/cdt_2")

In [ ]:
r = model.predict(_x_test)

In [ ]:
np.sum(r == _y_test)

In [ ]:
np.max(r)